In [7]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
import getpass
import os
import toml

secrets = toml.load(".streamlit/secrets.toml")
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_KEY"]

# Load Excel data
file_path = 'cluster.xlsx'
df = pd.read_excel(file_path)

organized_df = df.groupby('result').agg({
    'Text': lambda x: '\n'.join(x)  # Combine all text entries in each cluster with line breaks
}).reset_index()

# Count the number of rows in each cluster (create a separate DataFrame first)
counts_df = df.groupby('result').size().reset_index(name='count')

# Merge the counts back to the organized DataFrame
organized_df = organized_df.merge(counts_df, on='result', how='left')

# Check for repetition in each cluster
organized_df['repeat'] = organized_df.apply(
    lambda row: 1 if (row['Text'].split('\n').count(row['Text'].split('\n')[0]) / row['count']) > 0.9 else 0, axis=1)

organized_df = organized_df.head(10)

topic = "금융"

# Prepare the combined prompt using LangChain's PromptTemplate
combined_prompt_template = PromptTemplate(
    input_variables=["text", "topic"],
    template=(
        """다음 텍스트는 유저들이 챗봇에 작성한 질문들입니다.
        연관성을 기반으로 군집화된 질문들을 바탕으로 한국어로 짧게 질문들의 주제/목적을 생성하고,
        생성된 주제가 지정된 주제 '{topic}'와 얼마나 관련이 있는지 평가하십시오.
        관련성은 1(전혀 관련 없음)에서 4(매우 관련 있음) 사이의 숫자로 평가되어야 합니다.
        
        텍스트: {text}
        
        출력 형식 예시:
        주제: [생성된 주제]
        관련성: [1-4 숫자]
        """
    )
)

# Define an output parser to parse both the label and relevance from the response
output_parser = RegexParser(
    regex=r"주제:\s*(?P<label>.+?)\n관련성:\s*(?P<related>[1-4])",  # Regex to capture the label and relevance score
    output_keys=["label", "related"]
)

# Initialize ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# Create an LLMChain with the prompt template and output parser
chain = LLMChain(
    llm=llm,
    prompt=combined_prompt_template,
    output_parser=output_parser
)

# Function to get cluster labels and relevance ratings using a single API call
def generate_labels_and_relevance(texts, topic):
    results = []
    for text in texts:
        # Prepare input for the chain
        inputs = {"text": text, "topic": topic}
        # Use the invoke method to run the chain and parse the output
        parsed_output = chain.invoke(inputs)
        results.append((parsed_output["label"], int(parsed_output["related"])))
    return results

# Generate labels and relevance ratings using the LLMChain
results = generate_labels_and_relevance(organized_df['Text'].tolist(), topic)
organized_df['label'], organized_df['related'] = zip(*results)

# Ensure labels for noise points (-1) are NaN
organized_df.loc[organized_df['result'] == -1, ['label', 'related']] = pd.NA

# Save the result to a new Excel file
organized_df.to_excel('clustered.xlsx', index=False)
print("Clustered data with labels and relevance scores has been saved to 'clustered.xlsx'.")


Clustered data with labels and relevance scores has been saved to 'output_cluster_with_labels_and_relevance_chain.xlsx'.


In [9]:
import pandas as pd
from tqdm import tqdm
import hdbscan
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
from langchain_openai import OpenAIEmbeddings
import os
import toml

# Load secrets
secrets = toml.load(".streamlit/secrets.toml")
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_KEY"]

# Load Excel data
file_path = 'test.xlsx'
df = pd.read_excel(file_path)


# Initialize OpenAIEmbeddings with your API key
openai_embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"), model="text-embedding-3-small")

# Prepare text for embeddings
text = df['text'].tolist()

# Get OpenAI embeddings for the text
embeddings = openai_embeddings.embed_documents(text)

# Convert embeddings to a format suitable for further processing
vector = np.array(embeddings)

# Compute Distance Matrix
distance_matrix = pairwise_distances(vector, metric='cosine')

# HDBSCAN Clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2, metric='precomputed', cluster_selection_epsilon=0.01)
result = hdbscan_model.fit_predict(distance_matrix)

df['result'] = result

# Count the number of rows in each cluster
counts_df = df.groupby('result').size().reset_index(name='count')

# Create a DataFrame with cluster text
organized_df = df.groupby('result').agg({
    'text': lambda x: '\n'.join(x)  # Combine all text entries in each cluster with line breaks
}).reset_index()

# Merge the counts back to the organized DataFrame
organized_df = organized_df.merge(counts_df, on='result', how='left')

# Check for repetition in each cluster
organized_df['repeat'] = organized_df.apply(
    lambda row: 1 if (row['text'].split('\n').count(row['text'].split('\n')[0]) / row['count']) > 0.9 else 0, axis=1)

# Only process the first 10 clusters for the example
organized_df = organized_df.head(10)

# Define the topic for relevance rating
topic = "금융"  # You can replace this with a different topic if needed

# Prepare the combined prompt using LangChain's PromptTemplate
combined_prompt_template = PromptTemplate(
    input_variables=["text", "topic"],
    template=(
        """다음 텍스트는 유저들이 챗봇에 작성한 질문들입니다.
        연관성을 기반으로 군집화된 질문들을 바탕으로 한국어로 짧게 질문들의 주제/목적을 생성하고,
        생성된 주제가 지정된 주제 '{topic}'와 얼마나 관련이 있는지 평가하십시오.
        관련성은 1(전혀 관련 없음)에서 4(매우 관련 있음) 사이의 숫자로 평가되어야 합니다.
        
        텍스트: {text}
        
        출력 형식 예시:
        주제: [생성된 주제]
        관련성: [1-4 숫자]
        """
    )
)

# Define an output parser to parse both the label and relevance from the response
output_parser = RegexParser(
    regex=r"주제:\s*(?P<label>.+?)\n관련성:\s*(?P<related>[1-4])",  # Regex to capture the label and relevance score
    output_keys=["label", "related"]
)

# Initialize ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# Create an LLMChain with the prompt template and output parser
chain = LLMChain(
    llm=llm,
    prompt=combined_prompt_template,
    output_parser=output_parser
)

# Function to get cluster labels and relevance ratings using a single API call
def generate_labels_and_relevance(texts, topic):
    results = []
    for text in texts:
        # Prepare input for the chain
        inputs = {"text": text, "topic": topic}
        # Use the invoke method to run the chain and parse the output
        parsed_output = chain.run(inputs)
        results.append((parsed_output["label"], int(parsed_output["related"])))
    return results

# Generate labels and relevance ratings using the LLMChain
results = generate_labels_and_relevance(organized_df['text'].tolist(), topic)
organized_df['label'], organized_df['related'] = zip(*results)

# Ensure labels for noise points (-1) are NaN
organized_df.loc[organized_df['result'] == -1, ['label', 'related']] = pd.NA

# Save the result to a new DataFrame or Excel file
output_file = 'clustered.xlsx'
organized_df.to_excel(output_file, index=False)
print(f"Clustered data with labels and relevance scores has been saved to '{output_file}'.")


/Users/jaylee/Desktop/allproject1/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Clustered data with labels and relevance scores has been saved to 'clustered.xlsx'.
